In [1]:
import influxdb
from matplotlib import pyplot as plt
import pandas as pd
import db_secrets

In [2]:
HOST = db_secrets.HOST
PORT = db_secrets.PORT
USERNAME = db_secrets.USERNAME
PASSWORD = db_secrets.PASSWORD
DATABASE = db_secrets.DATABASE

Sensor_list = db_secrets.SensorIdList

QUERY_KEY = db_secrets.QUERY

client = influxdb.InfluxDBClient(HOST, PORT, USERNAME, PASSWORD, DATABASE, ssl=True, verify_ssl=True)

lux_list = []
timestamp_list = []
df_location_lux = []
df_location_lux_resampled = []

In [3]:
########################## Lux value queries #####################################

for index, value in enumerate(Sensor_list):
    
    location_lux = []
    location_timestamps = []
    
    location_vals = client.query('Select value from ' + QUERY_KEY + ' where time > now() - 60d and device_id  = ' + value)
    for val in location_vals:
        for vals in val:
            val_location = vals

            location_lux.append(val_location['value'])
            location_timestamps.append(val_location['time'])
    
    lux_list.append(location_lux)
    timestamp_list.append(location_timestamps)

In [4]:
############# DF creation ###################################

for index, value in enumerate(lux_list):
    location_lux_df = pd.DataFrame(
        {
            'time': timestamp_list[index],
            'Illumination(lx)': lux_list[index], 
        }
    )
    df_location_lux.append(location_lux_df)
    

In [ ]:
############################## Resampling data to 1 min intervals ##################################################

for index, value in enumerate(lux_list):
    df_location_lux[index]['time'] = pd.to_datetime(df_location_lux[index]['time'], format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce')
    df_location_lux[index].set_index('time', inplace=True)
    df_location_lux_resampled.append(df_location_lux[index].resample('T').first().fillna(method='ffill').reset_index())
    df_location_lux[index] = df_location_lux[index].reset_index()

In [ ]:
######################## Calculate light/dark time #########################

# df_list = []          # list of dfs
# location_thresholds = []  # choose thresholds for every area if they are different
# location_list = []        # list of locations


# for idx, df in enumerate(df_list):

#     threshold = location_thresholds[idx]
#     mask_on = df['lux_vals'] > threshold
#     mask_off = df['lux_vals'] <= threshold

#     total_on_duration = pd.Timedelta(seconds=0)
#     total_off_duration = pd.Timedelta(seconds=0)  

#     for i in range(len(df)):
#         if mask_on[i]:
#             if i < len(df) - 1:
#                 duration = df['time'][i + 1] - df['time'][i]
#                 total_on_duration += duration
#         elif mask_off[i]:
#             if i < len(df) - 1:
#                 duration = df['time'][i + 1] - df['time'][i]
#                 total_off_duration += duration

 
#     total_light_hours = total_on_duration.total_seconds() / 3600
#     total_dark_hours = total_off_duration.total_seconds() / 3600  

#     print("Total hours {} is in 'on' state: {}".format(location_list[idx], total_light_hours))
#     print("Total hours {} is in 'off' state: {}".format(location_list[idx], total_dark_hours))

In [37]:
######################## Saving to hdf5 files ##############################

for index, value in enumerate(df_location_lux):
    value.to_hdf('h5_data/direct/location_' + str(index + 1) + '_60days_historic_data.h5', key = 'data', mode='w')
    df_location_lux_resampled[index].to_hdf('h5_data/resampled/location_' + str(index + 1) + '_60days_minute_step_historic_data.h5', key = 'data', mode='w')

In [ ]:
################################### Plots #####################################

for index, value in enumerate(df_location_lux):
    value.plot(x = 'time', y = 'Illumination(lx)', title='Location ' + str(index + 1) + ' Lux Values for 60 days 1 min interval', xlabel='Time', ylabel='Illuminance (Lux)', color='green')
    df_location_lux_resampled[index].plot(x = 'time', y = 'Illumination(lx)', title='Location ' + str(index + 1) + ' Lux Values for 60 days', xlabel='Time', ylabel='Illuminance (Lux)', color='purple')

plt.show()